In [1]:
import re
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from collections import Counter
from razdel import sentenize
from razdel import tokenize as razdel_tokenize

Скачиваем корпус с опечатками

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

Создаём словарь из правильных словофрм и укороченных правильных словофрм (значение — правильная словоформа) и словарь частотностей

In [4]:
#скачиваем корпус(википедию)
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [5]:
WORDS = Counter(re.findall('\w+', corpus.lower()))
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [6]:
corpus = set(re.findall('\w+', corpus.lower()))

In [7]:
def delete(dictionary, word, depth=1): #можно конечно сделать изящнее (рекурсивно), но словарь надо создать один раз
    if len(word) > depth:
        if depth == 1:
                for i in range(len(word)):
                    current = word[:i] + word[(i+1):]
                    if current not in dictionary:
                        dictionary[current] = [word]
                    else:
                        dictionary[current].append(word)
        if depth == 2:
            for i in range(len(word)):
                for k in range(i+1, len(word)):
                    current = word[:i] + word[(i+1):k] + word[k+1:]
                    if current not in dictionary:
                        dictionary[current] = [word]
                    else:
                        dictionary[current].append(word)
#    return dictionary

In [8]:
#delete(delete({}, 'полк'), 'волк')

In [9]:
#delete(delete({}, 'полк', depth=2), 'волк', depth=2)

In [10]:
%%time
#проходим по корпусу и добавляем в словарь слова с удаленными буквами
vocab_0 = {} #правильные словоформы
vocab_1 = {} #удалена 1 буква
vocab_2 = {} #удалены 2 буквы
for word in corpus:
    vocab_0[word] = [word]
    delete(vocab_1, word, depth=1)
    delete(vocab_2, word, depth=2)
vocabs = [vocab_0, vocab_1, vocab_2]

Wall time: 39.7 s


In [11]:
vocab_1['олк']

['волк', 'уолк', 'толк', 'полк', 'фолк', 'иолк']

Создадим функцию, исправляющую опечатки

In [12]:
def choose(c_candidates):
    if len(c_candidates) == 0:
        return None
    choosen = max(c_candidates, key=P) 
    return choosen

In [13]:
def check_vocabs(word, vocabs, candidates, start_weight):
    for i in range(len(vocabs)): 
        vocab = vocabs[i]
        if word in vocab:
            weight = max(i, start_weight)
#            weight  = i + start_weight
            candidates[weight] += vocab[word]

In [14]:
def correction(word):
#    if word in vocab_0:
#        return word
    correct = word
    candidates = [[], [], [], [], []] 
    check_vocabs(word, vocabs, candidates, 0)
    for i in range(len(word)):
        current_1 = word[:i] + word[(i+1):]
        check_vocabs(current_1, vocabs, candidates, 1)
        for k in range(i+1, len(word)):
            current_2 = word[:i] + word[(i+1):k] + word[k+1:]
            check_vocabs(current_2, vocabs, candidates, 2)
    for c in candidates:
        if len(c) > 0:
            return choose(c)
    return correct

In [15]:
%%time
correction('олк')

Wall time: 18 ms


'полк'

In [16]:
%%time
correction('бронеттанспартёр')

Wall time: 0 ns


'бронетранспортёр'

Посмотрим на качество

In [17]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        predicted = correction(pair[1])
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [18]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8579420579420579
0.4405218726016884
0.07959113357069025


Теперь добавим триграммную модель: будем подавать в функцию correction не только слово, но и его соседа слева и справа, а вероятность слова в тексте будем домножать на вероятность появления этого слова с такими соседями

In [19]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [20]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [21]:
corpus_wiki = [['<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

In [22]:
trigrams = {} # словарь контекст : слово : сколько раз слово встречается в этом контексте

In [23]:
#собираем триграммы
for sent in corpus_wiki:
    for i in range(1 , len(sent)-1):
        token = sent[i]
        context = (sent[i-1], sent[i+1])
        if context not in trigrams:
            trigrams[context] = {}
            trigrams[context][token] = 1
        else:
            if token not in trigrams[context]:
                trigrams[context][token] = 1
            else:
                trigrams[context][token] += 1

In [24]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
#добавим токены начала и конца предложения
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = ['<start>'] + [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)] + ['<end>']
    tokens_2 = ['<start>'] + [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)] + ['<end>']
    
    return list(zip(tokens_1, tokens_2))

In [25]:
a = list(trigrams.keys())[101]
a

('к', 'в')

In [26]:
i = 0
for word in trigrams[a]:
    if i < 10:
        print(word, trigrams[a][word])
        i += 1
    else:
        break

западу 1
другу 1
постройке 1
постановке 1
vi 1
ней 22
возвращению 3
участию 23
гипоксии 1
примеру 14


In [27]:
#добавим в функцию выбора вероятность слова в контексте
def P_context(word, context):
    if context not in trigrams:
        return 1 #ничего не знаем про такой контекст 
    else:
        summa = 0
        trigram = trigrams[context]
        for w in trigram:
            summa += trigram[w]
        if word in trigram:
            return trigram[word]/summa
        else:
            return 1/(100*N) #слова не встречается в таком контексте, но если мы умножим на ноль вероятность для всех слов, мы не сможем выбрать нужное
def choose(c_candidates, context):
    if len(c_candidates) == 0:
        return None
    choosen = max(c_candidates, key=lambda x: P(x) * P_context(x, context)) 
    return choosen

In [28]:
#добавим контекст в функцию исправления
def correction(word, context):
#    if word in vocab_0:
#        return word
    correct = word
    candidates = [[], [], [], [], []] 
    check_vocabs(word, vocabs, candidates, 0)
    for i in range(len(word)):
        current_1 = word[:i] + word[(i+1):]
        check_vocabs(current_1, vocabs, candidates, 1)
        for k in range(i+1, len(word)):
            current_2 = word[:i] + word[(i+1):k] + word[k+1:]
            check_vocabs(current_2, vocabs, candidates, 2)
    for c in candidates:
        if len(c) > 0:
            return choose(c, context)
    return correct

Проверим качество новой функции

In [29]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for k in range(1, len(word_pairs)-1):
        pair = word_pairs[k]
        previous = word_pairs[k-1]
        next_ = word_pairs[k+1]
        context = (previous[1], next_[1])
        predicted = correction(pair[1], context)
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [30]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.859040959040959
0.44896392939370683
0.07959113357069025


Было вот так:  
0.8579420579420579  
0.4405218726016884  
0.07959113357069025  

Стало немного лучше. Попробуем не домножать на вероятность триграммы, а считать её приоритетной 

In [31]:
def choose(c_candidates, context):
    if len(c_candidates) == 0:
        return None
    choosen = max(c_candidates, key=lambda x: (P_context(x, context), P(x))) 
    return choosen

In [32]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for k in range(1, len(word_pairs)-1):
        pair = word_pairs[k]
        previous = word_pairs[k-1]
        next_ = word_pairs[k+1]
        context = (previous[1], next_[1])
        predicted = correction(pair[1], context)
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [33]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.859040959040959
0.44896392939370683
0.07959113357069025


А так уже ничего не изменилось